The first step is to download all dependencies we need. 

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert our Boston addresses into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map visualization library

!pip install geocoder
import geocoder

print('Libraries imported.')

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Libraries imported.


Next, we will create a JSON file for the wikipedia page to download the data and read it using pandas. 

In [2]:
#checking to see if file is downloaded
!wget -q -O 'seattle_data.json' https://en.wikipedia.org/wiki/List_of_neighborhoods_in_Seattle
print('Data downloaded!')

Data downloaded!


In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_neighborhoods_in_Seattle'
neighborhoods = pd.read_html(url, header = 0)[0]
neighborhoods

,Unnamed: 0,Neighborhood name,Within larger district,Annexed[41],Locator map,Street map,Image,Notes
0,1,North Seattle,Seattle,Various,NaN,NaN,NaN,North of the Lake Washington Ship Canal[42]
1,2,Broadview,North Seattle[42],1954[43],NaN,NaN,NaN,[44]
2,3,Bitter Lake,North Seattle[42],1954[43],NaN,NaN,NaN,[45]
3,4,North Beach / Blue Ridge,North Seattle[42],"1940,[43] 1954[43]",NaN,NaN,NaN,[46]
4,5,Crown Hill,North Seattle[42],"1907,[47] 1952,[43] 1954[43]",NaN,NaN,NaN,[48]
5,6,Greenwood,North Seattle[42],"1891,[43] 1954[43]",NaN,NaN,NaN,[49]
6,7,Northgate,North Seattle[42],various 1891-1954[43],NaN,NaN,NaN,[50]
7,8,Haller Lake,Northgate[50],1954[43],NaN,NaN,NaN,[51]
8,9,Pinehurst,Northgate[50],1953[43],NaN,NaN,NaN,[52]
9,10,North College Park (Licton Springs),Northgate[50],"1950, 1954[43]",NaN,NaN,NaN,[53]


In [4]:
neighborhoods = neighborhoods[["Neighborhood name", "Within larger district"]]
neighborhoods

,Neighborhood name,Within larger district
0,North Seattle,Seattle
1,Broadview,North Seattle[42]
2,Bitter Lake,North Seattle[42]
3,North Beach / Blue Ridge,North Seattle[42]
4,Crown Hill,North Seattle[42]
5,Greenwood,North Seattle[42]
6,Northgate,North Seattle[42]
7,Haller Lake,Northgate[50]
8,Pinehurst,Northgate[50]
9,North College Park (Licton Springs),Northgate[50]


In [5]:
data = pd.DataFrame() #copy of data frame to hold changes 
neighborhoods["Within larger district"] = neighborhoods["Within larger district"].str.replace('\d+', '') #removing footnote number
neighborhoods["Within larger district"] = neighborhoods["Within larger district"].apply(lambda y: y.replace('[','').replace(']','')) #removing footnote brackets

neighborhoods

C:\Users\Surbhi\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Surbhi\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Neighborhood name,Within larger district
0,North Seattle,Seattle
1,Broadview,North Seattle
2,Bitter Lake,North Seattle
3,North Beach / Blue Ridge,North Seattle
4,Crown Hill,North Seattle
5,Greenwood,North Seattle
6,Northgate,North Seattle
7,Haller Lake,Northgate
8,Pinehurst,Northgate
9,North College Park (Licton Springs),Northgate


In [6]:
neighborhoods.shape #checking columns,rows

(127, 2)

In [7]:
neighborhoods.columns = ['Neighborhood', "District"] #changing column names


Now, we will find the geographical location (latitude and longitude) of each neighborhood using Geocoder. 

In [8]:

for i, neighborhood in enumerate(neighborhoods["Neighborhood"]):
    latlong = None #holder variable for lat/long
    
    while(latlong is None): #making sure that I get the coordinates for all of our neighborhoods
        g = geocoder.arcgis('{}, Seattle, Washington'.format(neighborhood)) #getting latlong of each neighborhood
        latlong = g.latlng
    if latlong: 
        latitude = latlong[0] #holding lat of neighborhood i in variable 
        longitude = latlong[1] #holding long of neighborhood i in variable 
        
    neighborhoods.loc[i, "Latitude"] = latitude #adding lat at neighborhood i to datafame
    neighborhoods.loc[i, "Longitude"] = longitude #adding long at neighborhood i to datafame
    
neighborhoods

C:\Users\Surbhi\Anaconda3\lib\site-packages\pandas\core\indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\Surbhi\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,Neighborhood,District,Latitude,Longitude
0,North Seattle,Seattle,47.643724,-122.302937
1,Broadview,North Seattle,47.722380,-122.364980
2,Bitter Lake,North Seattle,47.718680,-122.350300
3,North Beach / Blue Ridge,North Seattle,47.700440,-122.384180
4,Crown Hill,North Seattle,47.695200,-122.374100
5,Greenwood,North Seattle,47.690820,-122.355290
6,Northgate,North Seattle,47.713100,-122.319300
7,Haller Lake,Northgate,47.723200,-122.338700
8,Pinehurst,Northgate,47.718940,-122.314000
9,North College Park (Licton Springs),Northgate,47.699140,-122.339680


I can now visualize the map of Seattle and look at the neighborhoods using Folium. 

In [9]:
#finding geographical coordinates of Seattle
address = 'Seattle, Washington'
geolocator = Nominatim(user_agent="seattle_explorer")
loc = geolocator.geocode(address)
Seattle_lat = loc.latitude
Seattle_long = loc.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(Seattle_lat, Seattle_long)) #printing coordinates

The geograpical coordinate of Manhattan are 47.6038321, -122.3300624.


In [10]:
map_seattle = folium.Map(location=[Seattle_lat, Seattle_long], zoom_start=9)

# adding markers to map
for lat, long, neigh, district in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Neighborhood'], neighborhoods['District']):
    label = '{}, {}'.format(neigh, district)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_seattle)  
    
map_seattle

I will now utilize Foursquare API to access nearby venues for each neighborhood. 

In [12]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
   
    #Code removed for private credentials
    VERSION = '20180605' # Foursquare API version
    LIMIT = 100
    venues_list=[]
    for name, lat, long in zip(names, latitudes, longitudes):
        print(name)
         
        # API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            long, 
            radius, 
            LIMIT)
            
        # GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return name, location, category for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            long, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

        #dataframe will have columns for nearby venues
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [13]:
#new dataframe with nearby venues for neighborhoods in Seattle
seattle_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude'])

North Seattle
Broadview
Bitter Lake
North Beach / Blue Ridge
Crown Hill
Greenwood
Northgate
Haller Lake
Pinehurst
North College Park (Licton Springs)
Maple Leaf
Lake City
Cedar Park
Matthews Beach
Meadowbrook
Olympic Hills
Victory Heights
Wedgwood
View Ridge
Sand Point
Roosevelt
Ravenna
Bryant
Windermere
Hawthorne Hills
Laurelhurst
University District (U District)
University Village
Wallingford
Northlake
Green Lake
Fremont
Phinney Ridge
Ballard
West Woodland
Whittier Heights
Adams
Sunset Hill
Loyal Heights
Central Seattle
Magnolia
Lawton Park
Briarcliff
Southeast Magnolia
Interbay
Queen Anne
North Queen Anne
East Queen Anne
Lower Queen Anne
West Queen Anne
Capitol Hill
Portage Bay[95] / Roanoke
Broadway
Pike-Pine Corridor / Pike/Pine[97][98][99]
Montlake
Stevens
Interlaken
Madison Valley
Renton Hill
Madison Park
Broadmoor
Lake Union
South Lake Union, Seattle
Cascade, Seattle
Westlake
Eastlake
Downtown
Denny Triangle
Belltown
Pike-Market
Central Business District
First Hill
Pioneer Squa

In [14]:
seattle_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,North Seattle,47.643724,-122.302937,Montlake Cut,47.647094,-122.304686,Canal
1,North Seattle,47.643724,-122.302937,Seattle Public Library - Montlake,47.640520,-122.302413,Library
2,North Seattle,47.643724,-122.302937,Cafe Lago,47.639698,-122.302256,Italian Restaurant
3,North Seattle,47.643724,-122.302937,Fuel Coffee - Montlake,47.639688,-122.302009,Coffee Shop
4,North Seattle,47.643724,-122.302937,Montlake Bicycle Shop,47.639380,-122.302340,Bike Shop


We can now see information on each nearby venue for each neighborhood. 

Next, we will group the data frame by category and then by neighborhood in order to look at the top venues of each neighborhood. 

In [15]:
#dataframe grouped by category

seattle_categories = pd.get_dummies(seattle_venues[['Venue Category']], prefix="", prefix_sep="")
seattle_categories["Neighborhood"] = seattle_venues["Neighborhood"]
columns = [seattle_categories.columns[-1]] + list(seattle_categories.columns[:-1])
seattle_categories = seattle_categories[columns]
seattle_categories.head()


,Yoga Studio,ATM,Accessories Store,African Restaurant,American Restaurant,Amphitheater,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Dealership,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Bike Shop,Bistro,Boat Rental,Boat or Ferry,Bookstore,Border Crossing,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Canal,Candy Store,Caribbean Restaurant,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comic Shop,Community College,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Credit Union,Creperie,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Dive Bar,Doctor's Office,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit,Eye Doctor,Fabric Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Floating Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,General Travel,German Restaurant,Gift Shop,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Herbs & Spices Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Inn,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Knitting Store,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Marijuana Dispensary,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Memorial Site,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,Nature Preserve,Neighborhood,New American Restaurant,Night Market,Nightclub,Noodle House,North Indian Restaurant,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Other Repair Shop,Outdoor Sculpture,Outdoor Supply Store,Park,Parking,Pawn Shop,Performing Arts Venue,Pet Service,Pet Store,Pharmacy,Photography Studio,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Post Office,Print Shop,Pub,Radio Station,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Restaurant,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Salsa Club,Sandwich Place,Scenic Lookout,School,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Ski Area,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Social Club,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Storage Facility,Supermarket,Supplement Shop,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Taiwanese R

In [16]:
#dataframe grouped by neighborhood
#averaging categories in neighborhoods
neighborhood_grouped = seattle_categories.groupby("Neighborhood").mean().reset_index()
neighborhood_grouped 

,Neighborhood,Yoga Studio,ATM,Accessories Store,African Restaurant,American Restaurant,Amphitheater,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Dealership,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Bike Shop,Bistro,Boat Rental,Boat or Ferry,Bookstore,Border Crossing,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Canal,Candy Store,Caribbean Restaurant,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comic Shop,Community College,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Credit Union,Creperie,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Dive Bar,Doctor's Office,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit,Eye Doctor,Fabric Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Floating Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,General Travel,German Restaurant,Gift Shop,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Herbs & Spices Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Inn,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Knitting Store,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Marijuana Dispensary,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Memorial Site,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,Nature Preserve,New American Restaurant,Night Market,Nightclub,Noodle House,North Indian Restaurant,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Other Repair Shop,Outdoor Sculpture,Outdoor Supply Store,Park,Parking,Pawn Shop,Performing Arts Venue,Pet Service,Pet Store,Pharmacy,Photography Studio,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Post Office,Print Shop,Pub,Radio Station,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Restaurant,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Salsa Club,Sandwich Place,Scenic Lookout,School,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Ski Area,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Social Club,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Storage Facility,Supermarket,Supplement Shop,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Taiwanese R

In [17]:
#viewing the top 10 categories for eavh neighborhood
num_top_venues = 10

for data in neighborhood_grouped ['Neighborhood']:
    print("----"+data+"----")
    top10 = neighborhood_grouped[neighborhood_grouped ['Neighborhood'] == data].T.reset_index()
    top10.columns = ['venue','freq']
    top10 = top10.iloc[1:]
    top10['freq'] = top10['freq'].astype(float)
    top10 = top10.round({'freq': 2}) #round to two decimal points
    print(top10.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adams----
              venue  freq
0      Burger Joint  0.08
1               Bar  0.05
2   Thai Restaurant  0.05
3            Bakery  0.05
4       Coffee Shop  0.05
5       Candy Store  0.03
6  Tapas Restaurant  0.03
7               Gym  0.03
8         Bookstore  0.03
9       Supermarket  0.03


----Alki Point----
                  venue  freq
0        Scenic Lookout   0.6
1                  Park   0.2
2     Convenience Store   0.2
3           Yoga Studio   0.0
4       Other Nightlife   0.0
5             Pawn Shop   0.0
6               Parking   0.0
7  Outdoor Supply Store   0.0
8     Outdoor Sculpture   0.0
9     Other Repair Shop   0.0


----Arbor Heights----
                  venue  freq
0                   Spa   1.0
1           Yoga Studio   0.0
2          Optical Shop   0.0
3               Parking   0.0
4                  Park   0.0
5  Outdoor Supply Store   0.0
6     Outdoor Sculpture   0.0
7     Other Repair Shop   0.0
8       Other Nightlife   0.0
9       Organic Grocery  

                venue  freq
0         Coffee Shop  0.10
1         Pizza Place  0.10
2        Dance Studio  0.05
3          Playground  0.05
4         Bus Station  0.05
5          Taco Place  0.05
6                Park  0.05
7  Mexican Restaurant  0.05
8    Greek Restaurant  0.05
9       Grocery Store  0.05


----Delridge----
                 venue  freq
0                 Park  0.31
1          Bus Station  0.15
2       Sandwich Place  0.08
3                  Gym  0.08
4          Coffee Shop  0.08
5         Soccer Field  0.08
6  Arts & Crafts Store  0.08
7           Skate Park  0.08
8             Bus Line  0.08
9      Organic Grocery  0.00


----Denny Triangle----
                  venue  freq
0                 Hotel  0.08
1           Coffee Shop  0.08
2             Juice Bar  0.05
3           Pizza Place  0.05
4    Italian Restaurant  0.04
5           Salad Place  0.03
6  Gym / Fitness Center  0.03
7         Grocery Store  0.03
8           Yoga Studio  0.03
9          Cycle Studio  0.03

                  venue  freq
0       Harbor / Marina  0.15
1            Playground  0.15
2            Steakhouse  0.08
3            Watch Shop  0.08
4              Cemetery  0.08
5                  Park  0.08
6  Ethiopian Restaurant  0.08
7           Pizza Place  0.08
8              Bus Stop  0.08
9                   Spa  0.08


----International District ("ID")----
                  venue  freq
0    Chinese Restaurant  0.12
1                Bakery  0.07
2              Tea Room  0.05
3              Bus Stop  0.05
4   Japanese Restaurant  0.05
5       Bubble Tea Shop  0.05
6  Taiwanese Restaurant  0.02
7  Gym / Fitness Center  0.02
8             Bookstore  0.02
9              Boutique  0.02


----Judkins Park----
                                      venue  freq
0                               Yoga Studio  0.07
1                                    Tunnel  0.07
2                                Food Truck  0.07
3                                      Food  0.07
4                          

9   Fried Chicken Joint  0.06


----Montlake----
                   venue  freq
0               Bus Stop  0.43
1          Grocery Store  0.14
2            Coffee Shop  0.14
3                Library  0.07
4     Italian Restaurant  0.07
5              Bike Shop  0.07
6    American Restaurant  0.07
7   Outdoor Supply Store  0.00
8  Performing Arts Venue  0.00
9              Pawn Shop  0.00


----Mount Baker----
                  venue  freq
0             Pet Store   0.2
1       Harbor / Marina   0.2
2                  Park   0.2
3                  Café   0.2
4           Pizza Place   0.2
5           Yoga Studio   0.0
6       Other Nightlife   0.0
7  Outdoor Supply Store   0.0
8     Outdoor Sculpture   0.0
9     Other Repair Shop   0.0


----North Admiral[153] / Admiral District----
                venue  freq
0         Coffee Shop  0.09
1                 Pub  0.06
2       Grocery Store  0.06
3         Pizza Place  0.06
4               Field  0.03
5                 Bar  0.03
6   Convenienc

9  Southern / Soul Food Restaurant  0.07


----Renton Hill----
                  venue  freq
0         Grocery Store  0.25
1        Cosmetics Shop  0.25
2           Gas Station  0.25
3     Other Repair Shop  0.25
4       Organic Grocery  0.00
5             Pawn Shop  0.00
6               Parking  0.00
7                  Park  0.00
8  Outdoor Supply Store  0.00
9     Outdoor Sculpture  0.00


----Riverview----
                  venue  freq
0                 Trail   0.4
1            Food Truck   0.2
2        Baseball Field   0.2
3                  Park   0.2
4           Yoga Studio   0.0
5       Organic Grocery   0.0
6               Parking   0.0
7  Outdoor Supply Store   0.0
8     Outdoor Sculpture   0.0
9     Other Repair Shop   0.0


----Roosevelt----
                       venue  freq
0                Coffee Shop  0.33
1  Latin American Restaurant  0.17
2                Gas Station  0.17
3               Optical Shop  0.17
4              Grocery Store  0.17
5                Yoga Studi

                  venue  freq
0                  Park   1.0
1           Yoga Studio   0.0
2          Optical Shop   0.0
3               Parking   0.0
4  Outdoor Supply Store   0.0
5     Outdoor Sculpture   0.0
6     Other Repair Shop   0.0
7       Other Nightlife   0.0
8       Organic Grocery   0.0
9           Opera House   0.0


----West Queen Anne----
                venue  freq
0         Coffee Shop  0.10
1              Bakery  0.10
2          Playground  0.10
3      Baseball Field  0.05
4            Dive Bar  0.05
5  Mexican Restaurant  0.05
6              Bistro  0.05
7   Convenience Store  0.05
8            Boutique  0.05
9       Grocery Store  0.05


----West Seattle----
                 venue  freq
0          Coffee Shop  0.17
1                Hotel  0.12
2  American Restaurant  0.07
3         Concert Hall  0.05
4       Sandwich Place  0.05
5              Theater  0.05
6       Chocolate Shop  0.02
7      Thai Restaurant  0.02
8                  Gym  0.02
9       Clothing Store 

In [18]:
#function to put these results into a data frame
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [19]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhood_venues_sorted = pd.DataFrame(columns=columns)
neighborhood_venues_sorted['Neighborhood'] = neighborhood_grouped['Neighborhood']

for ind in np.arange(neighborhood_grouped.shape[0]):
    neighborhood_venues_sorted.iloc[ind, 1:] = return_most_common_venues(neighborhood_grouped.iloc[ind, :], num_top_venues)

neighborhood_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adams,Burger Joint,Bakery,Bar,Thai Restaurant,Coffee Shop,Indian Restaurant,Ice Cream Shop,Sri Lankan Restaurant,Restaurant,Pub
1,Alki Point,Scenic Lookout,Park,Convenience Store,Women's Store,Field,Exhibit,Eye Doctor,Fabric Shop,Falafel Restaurant,Farmers Market
2,Arbor Heights,Spa,Women's Store,Filipino Restaurant,Exhibit,Eye Doctor,Fabric Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field
3,Atlantic,Coffee Shop,Vietnamese Restaurant,Bakery,Pharmacy,Seafood Restaurant,Sandwich Place,Bank,BBQ Joint,Park,Skate Park
4,Ballard,Mexican Restaurant,Cocktail Bar,Bakery,Italian Restaurant,Sushi Restaurant,Ice Cream Shop,Burger Joint,New American Restaurant,Thai Restaurant,Dessert Shop


Lastly, we will cluster these neighborhoods into groups by using K-Means clusstering algorithm. We will then investigate the top venues in each cluster to make a recommendation for which cluster is most ideal for a new resteraunt. 

In [20]:
kclusters = 5
seattle_clustering = neighborhood_grouped.drop("Neighborhood", 1)
kmeans = KMeans(n_clusters = kclusters, random_state = 0).fit(seattle_clustering)
kmeans.labels_[0:10]

array([0, 3, 0, 0, 0, 0, 0, 0, 2, 0])

In [21]:
neighborhood_venues_sorted.insert(0, "Cluster Labels", kmeans.labels_)
neighborhood_venues_sorted.head()
seattle_final = neighborhoods
seattle_final = seattle_final.join(neighborhood_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
seattle_final.head()

,Neighborhood,District,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North Seattle,Seattle,47.643724,-122.302937,0.0,Bus Stop,Coffee Shop,Grocery Store,Park,Library,American Restaurant,Italian Restaurant,Canal,Trail,Bike Shop
1,Broadview,North Seattle,47.722380,-122.364980,1.0,Trail,Concert Hall,Women's Store,Filipino Restaurant,Exhibit,Eye Doctor,Fabric Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant
2,Bitter Lake,North Seattle,47.718680,-122.350300,0.0,Marijuana Dispensary,Automotive Shop,Convenience Store,Hardware Store,Food Truck,Fast Food Restaurant,Furniture / Home Store,Steakhouse,Gas Station,Sushi Restaurant
3,North Beach / Blue Ridge,North Seattle,47.700440,-122.384180,3.0,Park,Café,Photography Studio,Garden Center,Women's Store,Field,Eye Doctor,Fabric Shop,Falafel Restaurant,Farmers Market
4,Crown Hill,North Seattle,47.695200,-122.374100,0.0,Pizza Place,Coffee Shop,Sports Bar,Greek Restaurant,Playground,Pet Store,Rock Club,Park,Sandwich Place,Bus Station


We can now seperate our dataframe by clusters.                      

<h3> Cluster 1 <h3>

In [22]:
seattle_final.loc[seattle_final['Cluster Labels'] == 0, seattle_final.columns[[1] + list(range(5, seattle_final.shape[1]))]]

,District,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Seattle,Bus Stop,Coffee Shop,Grocery Store,Park,Library,American Restaurant,Italian Restaurant,Canal,Trail,Bike Shop
2,North Seattle,Marijuana Dispensary,Automotive Shop,Convenience Store,Hardware Store,Food Truck,Fast Food Restaurant,Furniture / Home Store,Steakhouse,Gas Station,Sushi Restaurant
4,North Seattle,Pizza Place,Coffee Shop,Sports Bar,Greek Restaurant,Playground,Pet Store,Rock Club,Park,Sandwich Place,Bus Station
5,North Seattle,Coffee Shop,Mexican Restaurant,Bar,Bookstore,Spa,Pizza Place,Indian Restaurant,Comic Shop,Steakhouse,Sports Bar
6,North Seattle,Arts & Crafts Store,Yoga Studio,Jewelry Store,Sporting Goods Shop,Financial or Legal Service,Supermarket,Taco Place,Greek Restaurant,Pet Service,Kids Store
7,Northgate,Soccer Field,Chinese Restaurant,Vietnamese Restaurant,Bakery,Pool,Tennis Court,Gym,Juice Bar,Track,Athletics & Sports
8,Northgate,Pizza Place,Coffee Shop,Indian Restaurant,Bakery,Grocery Store,Liquor Store,Park,Rock Club,Sandwich Place,Smoke Shop
9,Northgate,Movie Theater,Asian Restaurant,Sandwich Place,Chinese Restaurant,Tanning Salon,Martial Arts School,Bus Station,Thai Restaurant,Coffee Shop,Shipping Store
10,Northgate,Optical Shop,Pharmacy,Arts & Crafts Store,Coffee Shop,Women's Store,Eye Doctor,Fabric Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant
12,Lake City,Brewery,Supermarket,Moving Target,Park,School,Farmers Market,Martial Arts School,Vietnamese Restaurant,General Entertainment,Middle Eastern Restaurant


<h3> Cluster 2 <h3>

In [23]:
seattle_final.loc[seattle_final['Cluster Labels'] == 1.0, seattle_final.columns[[1] + list(range(5, seattle_final.shape[1]))]]

,District,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North Seattle,Trail,Concert Hall,Women's Store,Filipino Restaurant,Exhibit,Eye Doctor,Fabric Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant
11,North Seattle,Trail,Women's Store,Ethiopian Restaurant,Exhibit,Eye Doctor,Fabric Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field


<h3> Cluster 3 <h3>

In [24]:
seattle_final.loc[seattle_final['Cluster Labels'] == 2, seattle_final.columns[[1] + list(range(5, seattle_final.shape[1]))]]

,District,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,North Seattle,Park,Bus Stop,Women's Store,Field,Eye Doctor,Fabric Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant
42,Magnolia,Bus Stop,Women's Store,Filipino Restaurant,Eye Doctor,Fabric Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Financial or Legal Service


<h3> Cluster 4 <h3>

In [25]:
seattle_final.loc[seattle_final['Cluster Labels'] == 3, seattle_final.columns[[1] + list(range(5, seattle_final.shape[1]))]]

,District,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,North Seattle,Park,Café,Photography Studio,Garden Center,Women's Store,Field,Eye Doctor,Fabric Shop,Falafel Restaurant,Farmers Market
13,Lake City,Park,Trail,Playground,Field,Event Space,Exhibit,Eye Doctor,Fabric Shop,Falafel Restaurant,Farmers Market
16,Lake City,Garden,Marijuana Dispensary,Beach,Playground,Park,Women's Store,Field,Eye Doctor,Fabric Shop,Falafel Restaurant
17,North Seattle,Park,Supermarket,Coffee Shop,Video Store,Steakhouse,Flower Shop,Exhibit,Eye Doctor,Fabric Shop,Falafel Restaurant
24,Windermere / North Seattle,Trail,Arts & Crafts Store,Nature Preserve,Playground,Park,Women's Store,Exhibit,Eye Doctor,Fabric Shop,Falafel Restaurant
28,North Seattle,Park,Seafood Restaurant,Boat or Ferry,Café,Food Truck,Gourmet Shop,Diner,Pizza Place,Bus Line,Tapas Restaurant
31,North Seattle,Park,Spa,Mexican Restaurant,General Travel,Gas Station,Music Store,Grocery Store,Karaoke Bar,Pharmacy,Coffee Shop
40,Central Seattle,Harbor / Marina,Park,Burger Joint,Scenic Lookout,Bus Station,Bus Line,Seafood Restaurant,Sporting Goods Shop,Financial or Legal Service,Fish & Chips Shop
60,Madison Park,Park,Ski Area,Soccer Field,Playground,Fast Food Restaurant,Exhibit,Eye Doctor,Fabric Shop,Falafel Restaurant,Farmers Market
87,Central Area,Park,Coffee Shop,Grocery Store,American Restaurant,Café,Playground,Pizza Place,Harbor / Marina,Women's Store,Farmers Market


<h3> Cluster 5 <h3>

In [26]:
seattle_final.loc[seattle_final['Cluster Labels'] == 4, seattle_final.columns[[1] + list(range(5, seattle_final.shape[1]))]]

,District,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
76,Downtown,Park,Women's Store,Filipino Restaurant,Eye Doctor,Fabric Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Financial or Legal Service
